In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import json

import pandas as pd
from rdkit import RDLogger
from pandarallel import pandarallel

import massspecgym.utils as utils

In [7]:
# Suppress RDKit warnings and errors
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

pandarallel.initialize(nb_workers=6, progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
with open(utils.hugging_face_download('molecules/MassSpecGym_retrieval_candidates_mass.json')) as f:
    cands_mass = json.load(f)
cands_mass = pd.DataFrame({'smiles': cands_mass.keys(), 'cands': cands_mass.values()})

with open(utils.hugging_face_download('molecules/MassSpecGym_retrieval_candidates_formula.json')) as f:
    cands_formula = json.load(f)
cands_formula = pd.DataFrame({'smiles': cands_formula.keys(), 'cands': cands_formula.values()})

## Number of positive samples among candidates

In [8]:
def n_pos(row):
    inchi_true = utils.smiles_to_inchi_key(row['smiles'])
    return sum(utils.smiles_to_inchi_key(c) == inchi_true for c in row['cands'])

cands_mass['n_pos'] = cands_mass.parallel_apply(n_pos, axis=1)
cands_mass['n_pos'].value_counts()

n_pos
1      24875
2       4306
3       1453
4        689
5        451
       ...  
103        1
81         1
129        1
85         1
64         1
Name: count, Length: 80, dtype: int64

In [9]:
def n_pos(row):
    inchi_true = utils.smiles_to_inchi_key(row['smiles'])
    return sum(utils.smiles_to_inchi_key(c) == inchi_true for c in row['cands'])

cands_formula['n_pos'] = cands_formula.parallel_apply(n_pos, axis=1)
cands_formula['n_pos'].value_counts()

n_pos
1      14161
2       7197
3       2619
4       2064
5       1150
       ...  
181        1
119        1
108        1
133        1
176        1
Name: count, Length: 139, dtype: int64